<a href="https://colab.research.google.com/github/EduardoArion/mlops-repo/blob/feature-gobcdmx/notebooks/02.Transformation/Merge_docs_INEGI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ahora me dedicaré a crear las "keys" para poder hacer el merge de los datos del censo y de geolocalización

Como ya los tenía arriba en el colab lo que hice fue guardarlos en mi Drive
Tendré que modificar este pedazo del código una vez que definamos donde vivirá nuestra información

In [1]:
!cp AGEB_fm.csv /content/drive/MyDrive/TFM/
!cp concat_catastral.csv /content/drive/MyDrive/TFM/


cp: cannot stat 'AGEB_fm.csv': No such file or directory
cp: cannot stat 'concat_catastral.csv': No such file or directory


In [3]:
!ls /content/drive/MyDrive/TFM/


ls: cannot access '/content/drive/MyDrive/TFM/': No such file or directory


In [1]:
import pandas as pd
import numpy as np
from google.colab import files

In [23]:
uploaded = files.upload()

Saving concat_catastral.csv to concat_catastral.csv


In [10]:
df_ageb = pd.read_csv("ageb_reducido.csv")
df_fm = pd.read_csv("09fm_con_coordenadas.csv")

/tmp/ipython-input-1344309785.py:2: DtypeWarning: Columns (11,12,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fm = pd.read_csv("09fm_con_coordenadas.csv")


In [11]:
df_fm.drop("CVEGEO", axis=1, inplace=True)

In [12]:
df_fm["key"] = (
    df_fm["CVE_ENT"].astype(str)
    + "_"
    + df_fm["CVE_MUN"].astype(str)
    + "_"
    + df_fm["CVE_LOC"].astype(str)
    + "_"
    + df_fm["CVE_AGEB"].astype(str)
    + "_"
    + df_fm["CVE_MZA"].astype(str)
)


In [13]:
df_ageb["key"] = (
    df_ageb["ENTIDAD"].astype(str)
    + "_"
    + df_ageb["MUN"].astype(str)
    + "_"
    + df_ageb["LOC"].astype(str)
    + "_"
    + df_ageb["AGEB"].astype(str)
    + "_"
    + df_ageb["MZA"].astype(str)
)

In [14]:
df_fm = df_fm.rename(columns={"CVE_ENT": "ENTIDAD", "CVE_MUN":"MUN", "CVE_LOC":"LOC", "CVE_AGEB":"AGEB", "CVE_MZA":"MZA"})

In [15]:
df_merged_m = pd.merge(
    df_ageb,
    df_fm,
    on="key",
    how="left",
    suffixes=("", "_fm")
)

In [10]:
df_merged_m[df_merged_m["key"] == '9_12_26_1528_48']

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,CVEREF2,TIPOVR2,NOMREF2,CVEVIAL3,CVESEG3,CVEREF3,TIPOVR3,NOMREF3,lon,lat
287923,9,Ciudad de México,12,Tlalpan,26,San Miguel Ajusco,1528,48,9,*,...,2.0,Calle,Tercera Francisco Peñuñuri,NaN,NaN,0.0,NaN,NaN,2.793927e+06,805161.164452
287924,9,Ciudad de México,12,Tlalpan,26,San Miguel Ajusco,1528,48,9,*,...,2.0,Calle,Tercera Francisco Peñuñuri,NaN,NaN,0.0,NaN,NaN,2.793896e+06,805159.413450
287925,9,Ciudad de México,12,Tlalpan,26,San Miguel Ajusco,1528,48,9,*,...,1.0,Calle,Tercera Cerrada de Francisco Peñuñuri,NaN,NaN,0.0,NaN,NaN,2.793908e+06,805126.426162


In [16]:
df_merged_m["key"]

,key
0,9_0_0_0000_0
1,9_2_0_0000_0
2,9_2_1_0000_0
3,9_2_1_0010_0
4,9_2_1_0010_1
...,...
375637,9_17_1_1524_25
375638,9_17_1_1524_25
375639,9_17_1_1524_25
375640,9_17_1_1524_25


In [17]:
df_merged_m.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,POBFEM,...,CVEREF2,TIPOVR2,NOMREF2,CVEVIAL3,CVESEG3,CVEREF3,TIPOVR3,NOMREF3,lon,lat
0,9,Ciudad de México,0,Total de la entidad Ciudad de México,0,Total de la entidad,0000,0,9209944,4805017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,Ciudad de México,2,Azcapotzalco,0,Total del municipio,0000,0,432205,227255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9,Ciudad de México,2,Azcapotzalco,1,Total de la localidad urbana,0000,0,432205,227255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,Ciudad de México,2,Azcapotzalco,1,Total AGEB urbana,0010,0,3183,1695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,Ciudad de México,2,Azcapotzalco,1,Azcapotzalco,0010,1,159,86,...,2.0,Calle,Física,1618.0,3.0,1.0,Avenida,Civilizaciones,2.792381e+06,837760.8518


In [18]:
print(len(df_merged_m), len(df_ageb), len(df_fm))

375642 68941 376258


Veo que estoy perdiendo datos al hacer mi merge, asi que averiguaré cuales son las keys que estoy perdiendo

In [19]:
keys_df1 = set(df_ageb["key"])
keys_df2 = set(df_fm["key"])

solo_df1 = keys_df1 - keys_df2
solo_df2 = keys_df2 - keys_df1

print("Keys solo en df1:", solo_df1)
print(len(solo_df1))
print("Keys solo en df2:", solo_df2)
print(len(solo_df2))


Keys solo en df1: {'9_7_1_209A_0', '9_7_1_5376_0', '9_10_1_0987_0', '9_7_1_4946_0', '9_7_1_287A_0', '9_17_1_0032_0', '9_17_1_0901_0', '9_11_110_1107_800', '9_9_24_0596_0', '9_7_1_5018_0', '9_15_1_0752_0', '9_5_1_2610_0', '9_5_1_3089_0', '9_7_1_2579_0', '9_10_1_2305_0', '9_16_1_0976_0', '9_11_21_1003_0', '9_11_1_1427_0', '9_7_1_3064_0', '9_3_1_1707_0', '9_5_1_1345_0', '9_5_1_2894_0', '9_14_1_0026_0', '9_12_26_2314_0', '9_7_1_4876_0', '9_6_1_1111_0', '9_12_1_2386_0', '9_7_1_4541_0', '9_6_1_0293_0', '9_4_1_0119_0', '9_8_1_0071_0', '9_17_1_1242_0', '9_12_1_0337_0', '9_5_1_1190_0', '9_16_1_1283_0', '9_7_1_1890_0', '9_17_1_023A_0', '9_3_1_1463_0', '9_17_0_0000_0', '9_13_1_1239_0', '9_15_1_1074_0', '9_6_1_0310_0', '9_10_1_0811_0', '9_14_1_0952_0', '9_5_1_0440_0', '9_10_1_0648_0', '9_7_1_5357_0', '9_5_1_3017_0', '9_13_1_025A_0', '9_5_1_2447_0', '9_7_1_4306_0', '9_5_1_0807_0', '9_2_1_0129_0', '9_11_1_1319_0', '9_10_1_181A_0', '9_17_1_0117_0', '9_17_1_009A_0', '9_7_1_2390_0', '9_7_1_3914_0', '9_

de mi df_ageb, estoy perdiendo todos los keys que terminan con 0, eso está bien, ya que estos son los que estan etiquetados como "totales de las localidades urbanas y municipales", que son datos cuantitativos, no datos puntuales

Ahora haré una pequeña exploración en las que estoy perdiendo en el df_mf

In [20]:
sorted(solo_df2)

['9_11_104_1198_1',
 '9_11_104_1198_2',
 '9_11_104_1198_3',
 '9_11_104_1198_4',
 '9_11_104_1198_5',
 '9_11_104_1198_6',
 '9_11_104_1198_7',
 '9_11_104_1198_8',
 '9_11_104_1198_9',
 '9_11_110_1107_8',
 '9_11_140_0471_1',
 '9_11_140_0471_10',
 '9_11_140_0471_11',
 '9_11_140_0471_12',
 '9_11_140_0471_13',
 '9_11_140_0471_14',
 '9_11_140_0471_15',
 '9_11_140_0471_16',
 '9_11_140_0471_17',
 '9_11_140_0471_18',
 '9_11_140_0471_19',
 '9_11_140_0471_2',
 '9_11_140_0471_20',
 '9_11_140_0471_21',
 '9_11_140_0471_22',
 '9_11_140_0471_3',
 '9_11_140_0471_4',
 '9_11_140_0471_5',
 '9_11_140_0471_6',
 '9_11_140_0471_7',
 '9_11_140_0471_8',
 '9_11_140_0471_9',
 '9_11_141_0471_1',
 '9_11_141_0471_2',
 '9_11_141_0471_3',
 '9_11_141_0471_4',
 '9_11_141_0471_5',
 '9_11_141_0471_6',
 '9_11_142_0471_1',
 '9_11_142_0471_2',
 '9_11_142_0471_3',
 '9_11_142_0471_4',
 '9_11_148_0471_1',
 '9_11_148_0471_2',
 '9_11_148_0471_3',
 '9_11_148_0471_4',
 '9_11_159_0471_1',
 '9_11_159_0471_2',
 '9_11_159_0471_3',
 '9_11_

In [21]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


patron = "9_9_46_0539_"

keys_cercanas = df_fm[df_fm["key"].str.startswith(patron)]
print(keys_cercanas)

keys_cercanas = df_ageb[df_ageb["key"].str.startswith(patron)]
print(keys_cercanas)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

        ENTIDAD  MUN  LOC  AGEB  MZA  CVEVIAL  CVESEG  CVEFT  \
4754          9    9   46  0539    6       15       1      2   
82050         9    9   46  0539    1       12       1      3   
82051         9    9   46  0539    1       12       1      4   
82052         9    9   46  0539    1       12       1      5   
82053         9    9   46  0539    1       13       1      7   
82054         9    9   46  0539    1       13       1      8   
82055         9    9   46  0539    1       13       1      9   
82056         9    9   46  0539    4        4       3      2   
82057         9    9   46  0539    4       11       1      3   
82058         9    9   46  0539    4        3       1      4   
82059         9    9   46  0539    7       17       1      2   
82060         9    9   46  0539    7       17       1      3   
82061         9    9   46  0539    8    70031   70031      1   
82062         9    9   46  0539    8        9       1      2   
82063         9    9   46  0539    8    

Ya analicé los datos que estoy perdiendo, y todos coinciden con claves de municipalidad (delegaciones para el DF), con las delegaciones de mayor marginalidad y dificil acceso. Asi que el hecho de que estos datos existan en el dataset de frente de manzana (df_dm), pero no esten enocntrando un para en el dataset de AGEB significa que no lograron accesar a esas áreas y por lo tanto no lograron hacer el levantamiento correspondiente. Por lo tanto, no será problematico para nosotros el que se pierdan en el merge.

Ahora voy a ddescargar el archivo para no tener que volcer a procesarlo

In [22]:
# Guardar en CSV
df_merged_m.to_csv("AGEB_fm.csv", index=False)

# Descargar
files.download("AGEB_fm.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>